<a href="https://colab.research.google.com/github/Vaelastraszz/Stack_OF/blob/master/Stack_Overflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
raw_post = pd.read_csv("/content/gdrive/My Drive/Stack_OF/QueryResults.csv")

In [0]:
pd.options.display.max_columns = None

In [5]:
raw_post.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,4,1,7.0,NaN,2008-07-31 21:42:52,NaN,605,39296.0,<p>I want to use a track-bar to change a form'...,8.0,NaN,6786713.0,Rich B,2018-07-02 17:55:27,2019-01-17 13:39:48,Convert Decimal to Double?,<c#><floating-point><type-conversion><double><...,13.0,2,46.0,NaN,2012-10-31 16:42:47
1,6,1,31.0,NaN,2008-07-31 22:08:08,NaN,274,17393.0,<p>I have an absolutely positioned <code>div</...,9.0,NaN,63550.0,Rich B,2016-03-19 06:05:48,2018-12-15 03:57:18,Percentage width child element in absolutely p...,<html><css><css3><internet-explorer-7>,6.0,0,11.0,NaN,NaN
2,7,2,NaN,4.0,2008-07-31 22:17:57,NaN,419,NaN,<p>An explicit cast to double like this isn't ...,9.0,NaN,4020527.0,NaN,2017-12-16 05:06:57,2017-12-16 05:06:57,NaN,NaN,NaN,0,NaN,NaN,NaN
3,9,1,1404.0,NaN,2008-07-31 23:40:59,NaN,1743,522602.0,<p>Given a <code>DateTime</code> representing ...,1.0,NaN,3956566.0,Rich B,2018-04-21 17:48:14,2018-07-25 11:57:14,How do I calculate someone's age in C#?,<c#><.net><datetime>,63.0,7,421.0,NaN,2011-08-16 19:40:43
4,11,1,1248.0,NaN,2008-07-31 23:55:37,NaN,1415,143330.0,<p>Given a specific <code>DateTime</code> valu...,1.0,NaN,6479704.0,user2370523,2017-06-04 15:51:19,2019-01-03 08:03:07,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,36.0,3,536.0,NaN,2009-09-04 13:15:59


In [6]:
raw_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40161 entries, 0 to 40160
Data columns (total 22 columns):
Id                       40161 non-null int64
PostTypeId               40161 non-null int64
AcceptedAnswerId         6576 non-null float64
ParentId                 32180 non-null float64
CreationDate             40161 non-null object
DeletionDate             0 non-null float64
Score                    40161 non-null int64
ViewCount                7981 non-null float64
Body                     40161 non-null object
OwnerUserId              38932 non-null float64
OwnerDisplayName         34788 non-null object
LastEditorUserId         15670 non-null float64
LastEditorDisplayName    9029 non-null object
LastEditDate             15860 non-null object
LastActivityDate         40161 non-null object
Title                    7981 non-null object
Tags                     7981 non-null object
AnswerCount              7981 non-null float64
CommentCount             40161 non-null int64
Favor

<h1> Nettoyage des données </h1>

In [0]:
post = raw_post[["Body", "Title", "Tags"]]

In [9]:
post.head()

,Body,Title,Tags
0,<p>I want to use a track-bar to change a form'...,Convert Decimal to Double?,<c#><floating-point><type-conversion><double><...
1,<p>I have an absolutely positioned <code>div</...,Percentage width child element in absolutely p...,<html><css><css3><internet-explorer-7>
2,<p>An explicit cast to double like this isn't ...,NaN,NaN
3,<p>Given a <code>DateTime</code> representing ...,How do I calculate someone's age in C#?,<c#><.net><datetime>
4,<p>Given a specific <code>DateTime</code> valu...,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...


In [7]:
post.isnull().sum().sort_values(ascending=False) / post.shape[0] * 100

Tags     80.127487
Title    80.127487
Body      0.000000
dtype: float64

In [8]:
post["Full_Post"] = np.where(post["Title"].isna(),post["Body"] , post["Title"] + " " + post["Body"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
post = post["Full_Post"]

<h2> Preprocessing des données avec NLTK + BeautifulSoup </h2>

<h3> BeautifulSoup </h3>

In [0]:
import nltk
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

In [0]:
def clean_code(post):
    for i in range(len(post)):
        post[i]=str(post[i]).replace(str(BeautifulSoup(post[i]).find("code")), "")
        post[i]= BeautifulSoup(post[i]).get_text()

In [0]:
liste_p = post.tolist()

In [0]:
clean_code(liste_p)

<h3> NLTK Processing </h3>

In [0]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

In [0]:
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize with POS Tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [0]:
def tokenize_list(liste_p):

    new_list = list()
    freq =  nltk.FreqDist()
    
    for i in range(len(liste_p)):
        
        new_list.append([lemmatizer.lemmatize(w.lower(),get_wordnet_pos(w.lower())) 
                                      for w in tokenizer.tokenize(liste_p[i]) if not w.lower() in stop_words and str.isalpha(w)== True ])
        
        freq += nltk.FreqDist(new_list[i])
        
    return new_list, freq

In [0]:
new_list,freq = tokenize_list(liste_p)

In [0]:
freq.most_common(50)

<h3> Tf-idf </h3>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

In [0]:
from nltk.tokenize.moses import MosesDetokenizer

detokenizer = MosesDetokenizer()

new_list = [detokenizer.detokenize(x, return_str=True) for x in new_list]

In [0]:
vectorizer =  TfidfVectorizer()

tf_idf = vectorizer.fit_transform(new_list)

In [69]:
tf_idf.shape

(40161, 43138)

<h3> Test Non negative matrix factorisation </h3>

In [0]:
from sklearn.decomposition import NMF

In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [73]:
no_topics = 30

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tf_idf)

no_top_words = 6
display_topics(nmf, vectorizer.get_feature_names(), no_top_words)

Topic 0:
would one get need time think
Topic 1:
php script mysql array language apache
Topic 2:
table column query select row index
Topic 3:
test unit write mock run integration
Topic 4:
file directory xml path folder config
Topic 5:
http com www aspx en microsoft
Topic 6:
net asp mvc framework vb ajax
Topic 7:
page javascript html browser cs tag
Topic 8:
server sql query client linq run
Topic 9:
string character format array convert match
Topic 10:
project visual studio build solution source
Topic 11:
use library instead also linq command
Topic 12:
window run command linux application xp
Topic 13:
try could get block follow problem
Topic 14:
class interface name public base create
Topic 15:
data xml access structure store form
Topic 16:
control source form event property button
Topic 17:
code write source generate compile compiler
Topic 18:
object reference collection create property memory
Topic 19:
like something look would could sound
Topic 20:
thread process lock pool multi run
To

<h2> Download packages NLTK </h2>

In [35]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('perluniprops')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.


True